In [ ]:
import os
import pypandoc
from pdf2docx  import parse

rootSubmissionFolder = 'Submissions'
textSubmissionsFolder = 'Text Submissions'
tempFolder = 'temp'
for assignmentFolder in os.listdir(rootSubmissionFolder):
    assignmentFolderPath = os.path.join(rootSubmissionFolder, assignmentFolder)
    assignmentID = assignmentFolder.split('_')[-1]
    if not os.path.exists(os.path.join(textSubmissionsFolder, assignmentID)):
            os.mkdir(os.path.join(textSubmissionsFolder, assignmentID))

    for submissionFile in os.listdir(assignmentFolderPath):
        submissionFilePath = os.path.join(rootSubmissionFolder, assignmentFolder, submissionFile)
        fileFormat = submissionFile.split('.')[-1].lower()

        if 'LATE' in submissionFile:
            userID = submissionFile.split('_')[2]
        else:
            userID = submissionFile.split('_')[1]

        savedFileName = userID+'.txt'
        savedFilePath = os.path.join(textSubmissionsFolder, assignmentID, savedFileName)
        if os.path.exists(savedFilePath):
            continue
        print(submissionFilePath)
        
        if fileFormat=='docx':
            output = pypandoc.convert_file(submissionFilePath, 'plain')
        elif fileFormat=='pdf':
            tempFilePath = os.path.join(tempFolder, 'tempFile.docx')
            parse(submissionFilePath, tempFilePath)
            output = pypandoc.convert_file(tempFilePath, 'plain')

        with open(savedFilePath, 'w') as textFile:
            textFile.write(output)
        


In [ ]:
textSubmissionsFolder = 'Text Submissions'
tempFolder = 'temp'
for assignmentID in os.listdir(textSubmissionsFolder):
    assignmentFolderPath = os.path.join(textSubmissionsFolder, assignmentID)

    for submissionFile in os.listdir(assignmentFolderPath):
        submissionFilePath = os.path.join(textSubmissionsFolder, assignmentID, submissionFile)
        with open(submissionFilePath) as textFile:
            submission = textFile.readlines()

        if len(submission) < 10:
            print(submissionFilePath)

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def num_tokens_from_string(string: str, encoding_name='cl100k_base') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
textSubmissionsFolder = 'Text Submissions'
tempFolder = 'temp'

tokenCountList = []
for assignmentID in os.listdir(textSubmissionsFolder):
    assignmentFolderPath = os.path.join(textSubmissionsFolder, assignmentID)

    for submissionFile in os.listdir(assignmentFolderPath):
        submissionFilePath = os.path.join(textSubmissionsFolder, assignmentID, submissionFile)
        with open(submissionFilePath) as textFile:
            submissionLines = textFile.readlines()
        submission = '\n'.join(submissionLines)

        tokenCountList.append(num_tokens_from_string(submission))

max(tokenCountList)

In [ ]:
from helper import *
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

def num_tokens_from_string(string: str, encoding_name='cl100k_base') -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

config = Config()
config.setFromEnv()

versionControl = 'V1'
promptVersion='P2'
fullName = f'{versionControl}-{promptVersion}'
config.setSaveDetails(fullName)
config.saveTemplatePrompt()

config.overWriteSave = False

config.simpleCourseName = 'Movement Science'

gradeRubricAssignmentDF = getGRAData(config)

for index, row in gradeRubricAssignmentDF.iterrows():
    fullPrompt = processTokenCount(row, config)
    if fullPrompt:
        if num_tokens_from_string(fullPrompt) > 8192:
            display(row)
